In [19]:
import pandas as pd
import numpy as np

# Import des différents CSV

data_pop_withA = pd.read_csv("P03/fr_population.csv", sep=",")
filtre_Agrégation_pop = data_pop_withA["Symbole"] != "A"
data_pop = data_pop_withA[filtre_Agrégation_pop]

data_veg_withA = pd.read_csv("P03/fr_vegetaux.csv", sep=",")
filtre_Agrégation_veg = data_veg_withA["Symbole"] != "A"
data_veg = data_veg_withA[filtre_Agrégation_veg]

data_ani_withA = pd.read_csv("P03/fr_animaux.csv", sep=";", encoding="cp1252") #lis fichier encodé cp1252
filtre_Agrégation_ani = data_ani_withA["Symbole"] != "A"
data_ani = data_ani_withA[filtre_Agrégation_ani]

data_sousnutri_withA = pd.read_csv("P03/fr_sousalimentation.csv", sep=";", encoding="ANSI", decimal=".") #lis fichier encodé ansi
filtre_Agrégation_sousnutri = data_sousnutri_withA["Code zone"] == 351
data_sousnutri_withoutA = data_sousnutri_withA[~filtre_Agrégation_sousnutri]

# # filtre pour les valeur 2013 en delete nan and <0.1
data_sousnutri_0nan = (data_sousnutri_withoutA["Valeur"] == "0") | (data_sousnutri_withoutA["Valeur"] == "<0.1") | (data_sousnutri_withoutA["Code année"] != 20122014) | (data_sousnutri_withoutA["Symbole"] == "NR")  | (data_sousnutri_withoutA["Symbole"] == "NV") 
data_sousnutri = data_sousnutri_withoutA[~data_sousnutri_0nan]


In [20]:
# Q19, code python pour créer les tables SQL demandé

# Data frame population

p3_pop_filtre = ["Code zone", "Zone", "Année", "Valeur"] #filtre colone
data_pop_clean = data_pop[p3_pop_filtre]
data_pop_clean["population"]= data_pop_clean["Valeur"]*1000
p3_bdd_pop = data_pop_clean.drop(columns=["Valeur"])


p3_bdd_pop.to_csv(path_or_buf="P03/fr_bdd_pop.csv", sep=';') # crée fichier CSV avec Dataframe
p3_bdd_pop



<ipython-input-20-30579e6bc89a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_pop_clean["population"]= data_pop_clean["Valeur"]*1000


,Code zone,Zone,Année,population
0,2,Afghanistan,2013,30552000
1,202,Afrique du Sud,2013,52776000
2,3,Albanie,2013,3173000
3,4,Algérie,2013,39208000
4,79,Allemagne,2013,82727000
...,...,...,...,...
170,236,Venezuela (République bolivarienne du),2013,30405000
171,237,Viet Nam,2013,91680000
172,249,Yémen,2013,24407000
173,251,Zambie,2013,14539000


In [21]:

# Data frame dispo_alim

p3_pop_merge_filtre = ["Code zone", "population"] #filtre colone pou préparer merge
p3_pop_merge = p3_bdd_pop[p3_pop_merge_filtre]


data_veg["origin"]="Végétales" #rajoute colon orgine (animal/végétal)
data_ani["origin"]="Animal"
p3_veg_ani = data_veg.append(data_ani) # rajoute animaux au végétaux


p3_veg_ani_clean = p3_veg_ani.drop(columns=["Domaine", "Code année","Code Domaine", "Symbole", "Description du Symbole"])

p3_veg_ani_pop = pd.merge(p3_pop_merge, #rajoute valeur pop pour calcul dalimtotal
                         p3_veg_ani_clean,              
             left_on=[ "Code zone"],
             right_on=["Code zone"])

# rajout collone dispo_alim_tonnes
p3_veg_ani_pop_Dalmitotal = p3_veg_ani_pop[p3_veg_ani_pop["Code Élément"] == 645]
p3_veg_ani_pop_Dalmitotal["dispo_alim_tonnes"] = (p3_veg_ani_pop_Dalmitotal["population"]*p3_veg_ani_pop_Dalmitotal["Valeur"]/1000) # /1000 pour passer de millier de tonne à tonne
p3_veg_ani_pop_Dalmitotal_clean = p3_veg_ani_pop_Dalmitotal.drop(columns=["population", "Valeur", "Code Élément", "Élément", "Unité"])


p3_veg_ani_pop_Dalmitotal_kcal = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 664], #rajoute valeur kcal
                         p3_veg_ani_pop_Dalmitotal_clean,  how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_pop_Dalmitotal_kcal_clean = p3_veg_ani_pop_Dalmitotal_kcal.rename(columns={"Valeur": "dispo_alim_kcal_p_j"})
p3_veg_ani_pop_Dalmitotal_kcal_clean2 = p3_veg_ani_pop_Dalmitotal_kcal_clean.drop(columns=["Code Élément", "Élément", "Unité"])




p3_veg_ani_pop_Dalmitotal_kcal_p = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 674], #rajoute valeur p
                         p3_veg_ani_pop_Dalmitotal_kcal_clean2,  how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_pop_Dalmitotal_kcal_p_clean = p3_veg_ani_pop_Dalmitotal_kcal_p.rename(columns={"Valeur": "dispo_prot"})
p3_veg_ani_pop_Dalmitotal_kcal_p_clean2 = p3_veg_ani_pop_Dalmitotal_kcal_p_clean.drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_pop_Dalmitotal_kcal_p_mg = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 684], #rajoute valeur mg
                         p3_veg_ani_pop_Dalmitotal_kcal_p_clean2,  how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_pop_Dalmitotal_kcal_p_mg_clean = p3_veg_ani_pop_Dalmitotal_kcal_p_mg.rename(columns={"Valeur": "dispo_mat_gr"})
p3_veg_ani_pop_Dalmitotal_kcal_p_mg_clean2 = p3_veg_ani_pop_Dalmitotal_kcal_p_mg_clean.drop(columns=["Code Élément", "Élément", "Unité"])

p3_bdd_dispo_alim = p3_veg_ani_pop_Dalmitotal_kcal_p_mg_clean2.fillna(-999)
p3_bdd_dispo_alim.to_csv(path_or_buf="P03/fr_bdd_dispo_alim.csv", sep=';') # crée fichier CSV avec Dataframe

p3_bdd_dispo_alim

# les valeur : -999, correspondront à des valeurs NULL dans la base SQL

<ipython-input-21-1def740ec860>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_veg["origin"]="Végétales" #rajoute colon orgine (animal/végétal)
<ipython-input-21-1def740ec860>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ani["origin"]="Animal"
<ipython-input-21-1def740ec860>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,Code zone,Zone,Code Produit,Produit,Année,dispo_mat_gr,origin,dispo_prot,dispo_alim_kcal_p_j,dispo_alim_tonnes
0,2,Afghanistan,2511,Blé,2013,4.69,Végétales,36.91,1369.0,4895346.96
1,2,Afghanistan,2805,Riz (Eq Blanchi),2013,0.27,Végétales,2.70,141.0,422228.64
2,2,Afghanistan,2513,Orge,2013,0.24,Végétales,0.79,26.0,89211.84
3,2,Afghanistan,2514,Maïs,2013,0.30,Végétales,0.56,21.0,76380.00
4,2,Afghanistan,2517,Millet,2013,0.02,Végétales,0.08,3.0,12220.80
...,...,...,...,...,...,...,...,...,...,...
14347,39,Tchad,2612,Citrons & Limes,2013,-999.00,Végétales,-999.00,-999.0,0.00
14348,39,Tchad,2613,Pamplemousse,2013,-999.00,Végétales,-999.00,-999.0,0.00
14349,222,Tunisie,2534,Racines nda,2013,-999.00,Végétales,-999.00,-999.0,0.00
14350,222,Tunisie,2616,Bananes plantains,2013,-999.00,Végétales,-999.00,-999.0,0.00


In [22]:
#  Data framme pour dispo_alim

# Merge en série pour rajouter les colonnes
p3_veg_ani_Dalimint = ((p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5301]).rename(columns={"Valeur": "Dalimint"})
                      ).drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_Dalimint_sem = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5527], #rajoute valeur semence
                         p3_veg_ani_Dalimint,  how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_Dalimint_sem_clean = (p3_veg_ani_Dalimint_sem.rename(columns={"Valeur": "semences"})).drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_Dalimint_sem_perte = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5123], #rajoute valeur perte
                         p3_veg_ani_Dalimint_sem_clean,   how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_Dalimint_sem_perte_clean = (p3_veg_ani_Dalimint_sem_perte.rename(columns={"Valeur": "pertes"})).drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_Dalimint_sem_perte_nour = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5142], #rajoute valeur nourriture
                         p3_veg_ani_Dalimint_sem_perte_clean,   how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_Dalimint_sem_perte_nour_clean = (p3_veg_ani_Dalimint_sem_perte_nour.rename(columns={"Valeur": "nourriture "})).drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_Dalimint_sem_perte_nour_alima = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5521], #rajoute valeur alim animaux
                         p3_veg_ani_Dalimint_sem_perte_nour_clean,   how = 'outer',             
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_Dalimint_sem_perte_nour_alima_clean = (p3_veg_ani_Dalimint_sem_perte_nour_alima.rename(columns={"Valeur": "alim_ani "})).drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5154], #rajoute valeur autres_utilisations
                         p3_veg_ani_Dalimint_sem_perte_nour_alima_clean,  how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse_clean = (p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse.rename(columns={"Valeur": "autres_utilisations "})).drop(columns=["Code Élément", "Élément", "Unité"])


p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse_trait = pd.merge(p3_veg_ani_clean[p3_veg_ani_clean["Code Élément"] == 5131], #rajoute valeur traitement
                         p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse_clean,  how = 'outer',              
             left_on=[ "Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"],
             right_on=["Code zone", "Zone", "Année", "Code Produit", "Produit", "origin"])
p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse_trait_clean = (p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse_trait.rename(columns={"Valeur": "traitement "})).drop(columns=["Code Élément", "Élément", "Unité", "origin"])

p3_bdd_equilibre_prod = p3_veg_ani_Dalimint_sem_perte_nour_alima_otheruse_trait_clean.fillna(-999)
p3_bdd_equilibre_prod.to_csv(path_or_buf="P03/fr_bdd_equilibre_prod.csv", sep=';') # crée fichier CSV avec Dataframe

p3_bdd_equilibre_prod.head(20)

# les valeur : -999, correspondront à des valeurs NULL dans la base SQL


,Code zone,Zone,Code Produit,Produit,Année,traitement,autres_utilisations,alim_ani,nourriture,pertes,semences,Dalimint
0,2,Afghanistan,2536,"Sucre, canne",2013,9.0,-999.0,81.0,-999.0,-999.0,-999.0,90.0
1,2,Afghanistan,2537,"Sucre, betterave",2013,15.0,-999.0,0.0,-999.0,-999.0,-999.0,15.0
2,2,Afghanistan,2557,Graines de tournesol,2013,3.0,-999.0,-999.0,-999.0,-999.0,0.0,3.0
3,2,Afghanistan,2559,Graines de coton,2013,26.0,-999.0,-999.0,-999.0,-999.0,1.0,27.0
4,2,Afghanistan,2561,Sésame,2013,4.0,-999.0,-999.0,16.0,1.0,1.0,21.0
5,2,Afghanistan,2563,Olives,2013,4.0,-999.0,-999.0,2.0,0.0,-999.0,7.0
6,2,Afghanistan,2570,"Plantes Oleiferes, Autre",2013,2.0,-999.0,-999.0,-999.0,0.0,0.0,3.0
7,202,Afrique du Sud,2511,Blé,2013,0.0,-999.0,37.0,3174.0,74.0,31.0,3316.0
8,202,Afrique du Sud,2805,Riz (Eq Blanchi),2013,0.0,-999.0,-999.0,919.0,0.0,0.0,920.0
9,202,Afrique du Sud,2513,Orge,2013,472.0,-999.0,19.0,5.0,7.0,6.0,509.0


In [23]:
p3_sousnutri = data_sousnutri.drop(columns=["Code Élément", "Élément", "Unité", "Domaine", "Année","Code Domaine", "Produit", "Note", "Symbole", "Description du Symbole"])

p3_sousnutri_2013 = p3_sousnutri[p3_sousnutri["Code année"] == 20122014]

# p3_sousnutri_2013["nb_personnes"] = float(p3_sousnutri_2013["Valeur"])





# converti valeur en float
p3_sousnutri_2013["nb_personnes"] = pd.to_numeric(p3_sousnutri_2013["Valeur"], errors='coerce')*1000000
p3_sousnutri_2013["Année"] = 2013

p3_bdd_sous_nutrition = p3_sousnutri_2013.drop(columns=["Code Produit", "Code année", "Valeur"]).fillna(-999)




p3_bdd_sous_nutrition.to_csv(path_or_buf="P03/fr_bdd_sous_nutrition.csv", sep=';') # crée fichier CSV avec Datafram

p3_bdd_sous_nutrition



,Code zone,Zone,nb_personnes,Année
0,2,Afghanistan,7900000.0,2013
5,202,Afrique du Sud,2600000.0,2013
10,3,Albanie,200000.0,2013
15,4,Algérie,1700000.0,2013
30,7,Angola,8100000.0,2013
...,...,...,...,...
995,236,Venezuela (République bolivarienne du),1900000.0,2013
1000,237,Viet Nam,10400000.0,2013
1005,249,Yémen,7200000.0,2013
1010,251,Zambie,7000000.0,2013
